In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import pandas as pd
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

Using TensorFlow backend.


In [2]:
data_path = 'data/'
images = 'images/'

In [3]:
droids_df = pd.read_pickle(f'{data_path}droids_df.pkl')

In [4]:
droids_df

,class,fileName,height,width,xmax,xmin,ymax,ymin,is_train,category
0,bb8,BB-8_1,512,288,208,46,512,15,True,1
1,bb8,BB-8_2,432,634,401,196,425,143,True,1
2,bb8,BB-8_3,480,480,443,23,480,155,False,1
3,bb8,BB-8_4,480,480,480,18,479,1,True,1
4,bb8,BB-8_5,332,800,442,332,332,208,False,1
...,...,...,...,...,...,...,...,...,...,...
322,background,background_8,169,298,297,1,168,1,True,3
323,background,background_11,183,275,274,1,182,1,True,3
324,background,background_6,168,300,299,1,167,1,True,3
325,background,background_5,168,300,299,1,167,1,True,3


In [5]:
num_classes = 3
SHAPE_WIDTH = 80
SHAPE_HEIGHT = 120

In [6]:
def resize_pack(pack):
    fx_ratio = SHAPE_WIDTH / pack.shape[1]
    fy_ratio = SHAPE_HEIGHT / pack.shape[0]    
    pack = cv2.resize(pack, (0, 0), fx=fx_ratio, fy=fy_ratio)
    return pack[0:SHAPE_HEIGHT, 0:SHAPE_WIDTH]

In [7]:
x, y, f = [], [], []
for fileName, is_train in droids_df[['fileName', 'is_train']].values:
    droids_rects = droids_df.loc[droids_df['fileName'] == fileName]
    rects_data = droids_rects[['category', 'xmin', 'ymin', 'xmax', 'ymax']]
    im = cv2.imread(f'{images}{fileName}.jpg')
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    for category, xmin, ymin, xmax, ymax in rects_data.values:
        if category == 0:
            continue
        pack = resize_pack(np.array(im[ymin:ymax, xmin:xmax]))
        x.append(pack)
        f.append(is_train)
        y.append(int(category) - 1)

error: /io/opencv/modules/imgproc/src/color.cpp:11079: error: (-215) scn == 3 || scn == 4 in function cvtColor


In [ ]:
plt.figure(figsize=(20,20))
for i in range(200):
    plt.subplot(10,20,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x[i], cmap=plt.cm.binary)
plt.show()

In [ ]:
# let's split the data to train/validation sets based on our is_train flag
x = np.array(x)
y = np.array(y)
f = np.array(f)
x_train, x_test, y_train, y_test = x[f], x[~f], y[f], y[~f]
# save test images
x_test_images = x_test

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
def lr_schedule(epoch):
#     lr = 1e-3
#     if epoch > 5:
#         lr *= 1e-1
#     print('Learning rate: ', lr)
#     return lr
    return 0.0005


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

def resnet_v1(input_shape, depth, num_classes=2):
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=x_train.shape[1:])
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

n = 3
version = 1
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2
model_type = 'ResNet%dv%d' % (depth, version)

model = resnet_v1(input_shape=x_train.shape[1:], depth=depth, num_classes=num_classes)
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)), metrics=['accuracy'])

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=5,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images
datagen.fit(x_train)

In [ ]:
batch_size = 30
epochs = 20
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    epochs=epochs, verbose=1, workers=4, 
                    callbacks=[LearningRateScheduler(lr_schedule)])

In [ ]:
# predictions = model.predict(x_test)

# class_names = np.array(['bb8','r2d2', 'none'])
# for i in range(len(x_test)):
#     plt.imshow(x_test[i], cmap=plt.cm.binary)
#     print(round(predictions[i][0], 4), round(predictions[i][1], 4), round(predictions[i][2], 4))
        
#     if predictions[i][0] == max(predictions[i]):
#         plt.xlabel(class_names[0])
#     elif predictions[i][1] == max(predictions[i]): plt.xlabel(class_names[1])
#     else: plt.xlabel(class_names[2])

#     plt.show()

In [ ]:
loss, acc = model.evaluate(x_test, y_test, verbose=1)

print('Test loss:', loss)
print('Test accuracy:', acc)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

In [ ]:
# model.save('my_model/model_92.h5')